In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime 
import pyodbc
import sqlalchemy as sal


In [2]:
def extract_data():
    url = 'https://www.psx.com.pk/market-summary/#main'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    container = soup.find('div', class_='col-sm-12 tab-pane inner-content-table automobile-div active')
    rows = container.find_all('tr', class_='red-text-td')
    dictionary = {
    'titles':[],
    'ldcp': [],
    'opens': [],
    'high': [],
    'low': [],
    'current': [],
    'change': [],
    'volume': [],
    'scrap_time':[]
    }

    for row in rows:
        a = row.find_all('td')
        dictionary['titles'].append(a[0].text.strip())
        dictionary['ldcp'].append(a[1].text.strip())
        dictionary['opens'].append(a[2].text.strip())
        dictionary['high'].append(a[3].text.strip())
        dictionary['low'].append(a[4].text.strip())
        dictionary['current'].append(a[5].text.strip())
        dictionary['change'].append(a[6].text.strip())
        dictionary['volume'].append(a[7].text.strip())
        dictionary['scrap_time'].append(datetime.datetime.now())
    print(f'✅ Data of {len(dictionary['titles'])} lines extracted successfully.')
    return dictionary

In [3]:
url = 'https://www.psx.com.pk/market-summary/#main'

In [4]:
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
container = soup.find('div', class_='col-sm-12 tab-pane inner-content-table automobile-div active')


In [7]:
rows = container.find_all('tr', class_='red-text-td')

In [8]:
dictionary = {
    'titles':[],
    'ldcp': [],
    'opens': [],
    'high': [],
    'low': [],
    'current': [],
    'change': [],
    'volume': [],
    'scrap_time':[]
}

for row in rows:
    a = row.find_all('td')
    dictionary['titles'].append(a[0].text.strip())
    dictionary['ldcp'].append(a[1].text.strip())
    dictionary['opens'].append(a[2].text.strip())
    dictionary['high'].append(a[3].text.strip())
    dictionary['low'].append(a[4].text.strip())
    dictionary['current'].append(a[5].text.strip())
    dictionary['change'].append(a[6].text.strip())
    dictionary['volume'].append(a[7].text.strip())
    dictionary['scrap_time'].append(datetime.datetime.now())

In [9]:
def transform_data(dictionary):
    df = pd.DataFrame(dictionary)
    df = df.replace(',', '', regex=True)
    df = df.replace('', pd.NA)
    type_conv = {
    'ldcp': float,
    'opens': float,
    'high': float,
    'low': float,
    'current': float,
    'change': float,
    'volume': float
    }

    for i in type_conv:
        df[i] = pd.to_numeric(df[i], errors='coerce')
    df['day_range'] = df['high'] - df['low']
    df['volatility_perc'] = ((df['high'] - df['low'])/ df['opens'])*100
    df_volatile = df.nlargest(5, 'volatility_perc')
    df_high_performance = df.nlargest(5, 'change')
    df_low_performance = df.nsmallest(5, 'change')
    return df, df_volatile, df_high_performance, df_low_performance

In [10]:
df = pd.DataFrame(dictionary)
df

,titles,ldcp,opens,high,low,current,change,volume,scrap_time
0,Sazgar Engineering,"1,270.82",1271.0,1276.79,1251.0,1258.0,-12.82,"81,693",2025-07-21 13:51:27.474611
1,Bal.Wheels,150.61,151.98,151.98,148.0,148.5,-2.11,"2,463",2025-07-21 13:51:27.474611
2,Ghandhara Tyre,47.65,47.65,48.0,47.0,47.28,-0.37,"217,768",2025-07-21 13:51:27.474611
3,Loads Limited,14.91,15.04,15.04,14.3,14.5,-0.41,"1,952,394",2025-07-21 13:51:27.474611
4,Thal Limited,431.43,430.0,430.0,423.01,425.0,-6.43,"1,308",2025-07-21 13:51:27.474611
...,...,...,...,...,...,...,...,...,...
357,P.N.S.C,398.09,400.0,401.0,395.0,396.05,-2.04,"11,043",2025-07-21 13:51:27.502311
358,Pak.Int.Container,44.66,44.74,44.9,43.74,44.22,-0.44,"127,106",2025-07-21 13:51:27.502311
359,Secure Logistics Gro,16.72,16.8,16.84,16.25,16.46,-0.26,"969,323",2025-07-21 13:51:27.502311
360,Punjab Oil,232.53,227.2,238.0,225.02,229.75,-2.78,"4,727",2025-07-21 13:51:27.502311


In [11]:
df.dtypes

titles                object
ldcp                  object
opens                 object
high                  object
low                   object
current               object
change                object
volume                object
scrap_time    datetime64[ns]
dtype: object

In [12]:
df = df.replace(',', '', regex=True)
df = df.replace('', pd.NA)

In [13]:
type_conv = {
    'ldcp': float,
    'opens': float,
    'high': float,
    'low': float,
    'current': float,
    'change': float,
    'volume': float
}

for i in type_conv:
    df[i] = pd.to_numeric(df[i], errors='coerce')

In [14]:
df.dtypes

titles                object
ldcp                 float64
opens                float64
high                 float64
low                  float64
current              float64
change               float64
volume                 int64
scrap_time    datetime64[ns]
dtype: object

In [15]:
df['change'].nlargest(5)

35    -0.01
89    -0.01
228   -0.01
267   -0.01
271   -0.01
Name: change, dtype: float64

In [16]:
df['opens'].nlargest(5)

207    31015.32
210     1699.00
7       1532.00
64      1338.00
212     1310.00
Name: opens, dtype: float64

In [17]:
df['day_range'] = df['high'] - df['low']

In [18]:
df

,titles,ldcp,opens,high,low,current,change,volume,scrap_time,day_range
0,Sazgar Engineering,1270.82,1271.00,1276.79,1251.00,1258.00,-12.82,81693,2025-07-21 13:51:27.474611,25.79
1,Bal.Wheels,150.61,151.98,151.98,148.00,148.50,-2.11,2463,2025-07-21 13:51:27.474611,3.98
2,Ghandhara Tyre,47.65,47.65,48.00,47.00,47.28,-0.37,217768,2025-07-21 13:51:27.474611,1.00
3,Loads Limited,14.91,15.04,15.04,14.30,14.50,-0.41,1952394,2025-07-21 13:51:27.474611,0.74
4,Thal Limited,431.43,430.00,430.00,423.01,425.00,-6.43,1308,2025-07-21 13:51:27.474611,6.99
...,...,...,...,...,...,...,...,...,...,...
357,P.N.S.C,398.09,400.00,401.00,395.00,396.05,-2.04,11043,2025-07-21 13:51:27.502311,6.00
358,Pak.Int.Container,44.66,44.74,44.90,43.74,44.22,-0.44,127106,2025-07-21 13:51:27.502311,1.16
359,Secure Logistics Gro,16.72,16.80,16.84,16.25,16.46,-0.26,969323,2025-07-21 13:51:27.502311,0.59
360,Punjab Oil,232.53,227.20,238.00,225.02,229.75,-2.78,4727,2025-07-21 13:51:27.502311,12.98


In [19]:
df['volatility_perc'] = ((df['high'] - df['low'])/ df['opens'])*100

In [20]:
df

,titles,ldcp,opens,high,low,current,change,volume,scrap_time,day_range,volatility_perc
0,Sazgar Engineering,1270.82,1271.00,1276.79,1251.00,1258.00,-12.82,81693,2025-07-21 13:51:27.474611,25.79,2.029111
1,Bal.Wheels,150.61,151.98,151.98,148.00,148.50,-2.11,2463,2025-07-21 13:51:27.474611,3.98,2.618766
2,Ghandhara Tyre,47.65,47.65,48.00,47.00,47.28,-0.37,217768,2025-07-21 13:51:27.474611,1.00,2.098636
3,Loads Limited,14.91,15.04,15.04,14.30,14.50,-0.41,1952394,2025-07-21 13:51:27.474611,0.74,4.920213
4,Thal Limited,431.43,430.00,430.00,423.01,425.00,-6.43,1308,2025-07-21 13:51:27.474611,6.99,1.625581
...,...,...,...,...,...,...,...,...,...,...,...
357,P.N.S.C,398.09,400.00,401.00,395.00,396.05,-2.04,11043,2025-07-21 13:51:27.502311,6.00,1.500000
358,Pak.Int.Container,44.66,44.74,44.90,43.74,44.22,-0.44,127106,2025-07-21 13:51:27.502311,1.16,2.592758
359,Secure Logistics Gro,16.72,16.80,16.84,16.25,16.46,-0.26,969323,2025-07-21 13:51:27.502311,0.59,3.511905
360,Punjab Oil,232.53,227.20,238.00,225.02,229.75,-2.78,4727,2025-07-21 13:51:27.502311,12.98,5.713028


In [21]:
df['volatility_perc'].nlargest(5)

301    19.331372
305    16.375727
356    15.804765
328    15.491329
326    14.505119
Name: volatility_perc, dtype: float64

In [22]:
df_volatile = df.nlargest(5, 'volatility_perc')

In [23]:
df_high_performance = df.nlargest(5, 'change')

In [24]:
df_low_performance = df.nsmallest(5, 'change')

In [28]:
def create_connection():
    conn_str = 'mssql://Kabir-Khan-PC/airflowProject?driver=ODBC+DRIVER+17+FOR+SQL+SERVER'

    try:
        engine = sal.create_engine(conn_str)
        conn = engine.connect()
        print("✅ Connected to SQL Server successfully!")
        return conn
    except Exception as e:
        print(f"❌ Error connecting to SQL Server: {e}")
        return None

def insert_data():
    conn = create_connection()
    if not conn:
        return
    try:
        df.to_sql(f'psx_info', con=conn, index=True, index_label='id', if_exists='append')
        print(f"✅ Inserted {len(df)} rows into database.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")

    
    try:
        df_volatile.to_sql(f'psx_volatile', con=conn, index=True, index_label='id',  if_exists='append')
        print(f"✅ Inserted {len(df_volatile)} rows into database.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")

    
    try:
        df_high_performance.to_sql(f'psx_high_performance', con=conn, index=True, index_label='id', if_exists='append')
        print(f"✅ Inserted {len(df_high_performance)} rows into database.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")

    
    try:
        df_low_performance.to_sql(f'psx_low_performance', con=conn, index=True, index_label='id', if_exists='append')
        print(f"✅ Inserted {len(df_low_performance)} rows into database.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")    
    

In [ ]:
extract_data()


In [ ]:
transform_data(dictionary)

In [29]:
insert_data()

✅ Connected to SQL Server successfully!
✅ Inserted 362 rows into database.
✅ Inserted 5 rows into database.
✅ Inserted 5 rows into database.
✅ Inserted 5 rows into database.
